# 간단한 RNN으로 텍스트 생성하기
------------------------------
reference: [RNN을 이용한 텍스트 생성](wikidocs.net/45101)

## 데이터 전처리

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

text="경마장에 있는 말이 뛰고 있다\n그의 말이 법이다\n가는 말이 고와야 오는 말이 곱다\n"

In [2]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : {}'.format(vocab_size))
print(t.word_index)

단어 집합의 크기 : 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


훈련 데이터로 재구성

In [3]:
sequences = []

# 줄바꿈을 기준으로 문장 토큰화
print(text.split('\n'))
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: {}'.format(len(sequences)))
print(sequences)

['경마장에 있는 말이 뛰고 있다', '그의 말이 법이다', '가는 말이 고와야 오는 말이 곱다', '']
[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]
[]
학습에 사용할 샘플의 개수: 11
[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


학습에 사용할 샘플은 길이가 6으로 가장 길다. 전체 샘플을 6으로 패딩해준다.

In [4]:
# 길이 6으로 패딩
max_len=max(len(l) for l in sequences)
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


학습 진행을 위해 훈련 데이터를 x, y로 분리하자.

In [5]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:,-1]
# one hot encoding for y
y = to_categorical(y, num_classes=vocab_size)
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 모델 설계하기

간단한 RNN 모델을 만들어 학습을 진행하자.

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

rnn_model = Sequential()
# y를 분리해내었으므로 input_length=max_len-1 로 줄었다
rnn_model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# 32 hidden state를 가지는 바닐라 RNN
rnn_model.add(SimpleRNN(32))
rnn_model.add(Dense(vocab_size, activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.fit(X, y, epochs=150, verbose=2)

C:\Users\SHIN-DESKTOP\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/150
 - 3s - loss: 2.6577 - acc: 0.0909
Epoch 2/150
 - 0s - loss: 2.6107 - acc: 0.0909
Epoch 3/150
 - 0s - loss: 2.5582 - acc: 0.0909
Epoch 4/150
 - 0s - loss: 2.5049 - acc: 0.0909
Epoch 5/150
 - 0s - loss: 2.4525 - acc: 0.2727
Epoch 6/150
 - 0s - loss: 2.4021 - acc: 0.2727
Epoch 7/150
 - 0s - loss: 2.3543 - acc: 0.2727
Epoch 8/150
 - 0s - loss: 2.3092 - acc: 0.3636
Epoch 9/150
 - 0s - loss: 2.2670 - acc: 0.3636
Epoch 10/150
 - 0s - loss: 2.2278 - acc: 0.3636
Epoch 11/150
 - 0s - loss: 2.1913 - acc: 0.3636
Epoch 12/150
 - 0s - loss: 2.1574 - acc: 0.3636
Epoch 13/150
 - 0s - loss: 2.1259 - acc: 0.3636
Epoch 14/150
 - 0s - loss: 2.0966 - acc: 0.3636
Epoch 15/150
 - 0s - loss: 2.0690 - acc: 0.3636
Epoch 16/150
 - 0s - loss: 2.0431 - acc: 0.3636
Epoch 17/150
 - 0s - loss: 2.0185 - acc: 0.3636
Epoch 18/150
 - 0s - loss: 1.9950 - acc: 0.3636
Epoch 19/150
 - 0s - loss: 1.9724 - acc: 0.3636
Epoch 20/150
 - 0s - loss: 1.9504 - acc: 0.3636
Epoch 21/150
 - 0s - loss: 1.9290 - acc: 0.3636
E

학습 결과를 알아보기 위해 간단한 문장을 출력하는 함수를 만들어본다.

In [28]:
def sentence_generation(model, t, current_word, n, max_len): 
    # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    init_word = current_word 
    sentence = ''
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            # 해당 단어가 예측 단어이므로 break
            if index == result: 
                break
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word
        sentence = sentence + ' ' + word 

    sentence = init_word + sentence
    return sentence

예측횟수를 적절히 주면 제대로 된 출력결과는 아래와 같다.

In [9]:
print(sentence_generation(rnn_model, t, '경마장에', 4, max_len))
print(sentence_generation(rnn_model, t, '그의', 2, max_len))
print(sentence_generation(rnn_model, t, '가는', 5, max_len))

encoded word: [[0 0 0 0 2]]
predicted word: [3]
encoded word: [[0 0 0 2 3]]
predicted word: [1]
encoded word: [[0 0 2 3 1]]
predicted word: [4]
encoded word: [[0 2 3 1 4]]
predicted word: [5]
경마장에 있는 말이 뛰고 있다
encoded word: [[0 0 0 0 6]]
predicted word: [1]
encoded word: [[0 0 0 6 1]]
predicted word: [7]
그의 말이 법이다
encoded word: [[0 0 0 0 8]]
predicted word: [1]
encoded word: [[0 0 0 8 1]]
predicted word: [9]
encoded word: [[0 0 8 1 9]]
predicted word: [10]
encoded word: [[ 0  8  1  9 10]]
predicted word: [1]
encoded word: [[ 8  1  9 10  1]]
predicted word: [11]
가는 말이 고와야 오는 말이 곱다


예측횟수를 기존 문장보다 많이 주거나, 사전에 학습하지 않은 단어를 주었을 때의 결과이다.

In [10]:
print(sentence_generation(rnn_model, t, '경마장에', 5, max_len))
print(sentence_generation(rnn_model, t, '그의', 3, max_len))
print(sentence_generation(rnn_model, t, '가는', 6, max_len))
print(sentence_generation(rnn_model, t, '그녀의', 3, max_len))

encoded word: [[0 0 0 0 2]]
predicted word: [3]
encoded word: [[0 0 0 2 3]]
predicted word: [1]
encoded word: [[0 0 2 3 1]]
predicted word: [4]
encoded word: [[0 2 3 1 4]]
predicted word: [5]
encoded word: [[2 3 1 4 5]]
predicted word: [11]
경마장에 있는 말이 뛰고 있다 곱다
encoded word: [[0 0 0 0 6]]
predicted word: [1]
encoded word: [[0 0 0 6 1]]
predicted word: [7]
encoded word: [[0 0 6 1 7]]
predicted word: [11]
그의 말이 법이다 곱다
encoded word: [[0 0 0 0 8]]
predicted word: [1]
encoded word: [[0 0 0 8 1]]
predicted word: [9]
encoded word: [[0 0 8 1 9]]
predicted word: [10]
encoded word: [[ 0  8  1  9 10]]
predicted word: [1]
encoded word: [[ 8  1  9 10  1]]
predicted word: [11]
encoded word: [[ 1  9 10  1 11]]
predicted word: [7]
가는 말이 고와야 오는 말이 곱다 법이다
encoded word: [[0 0 0 0 0]]
predicted word: [1]
encoded word: [[0 0 0 0 1]]
predicted word: [1]
encoded word: [[0 0 0 1 1]]
predicted word: [7]
그녀의 말이 말이 법이다


# LSTM을 이용하여 텍스트 생성하기
----------------------------------

## 데이터 전처리

In [11]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [12]:
df = pd.read_csv('.\\nyt-comments\\ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [13]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [14]:
# is there any Null value
df['headline'].isnull().values.any()

False

In [15]:
# store headline
headline = []
headline.extend(list(df.headline.values)) 
print(headline[:5])

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell', 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.', 'The New Noma, Explained', 'Unknown', 'Unknown']


Null 값은 없지만 unknown 값이 있으므로 이런 노이즈는 미리 제거해주자.

In [16]:
print('노이즈 제거 전: {}'.format(len(headline)))
headline = [n for n in headline if n != "Unknown"]
print('노이즈 제거 후: {}'.format(len(headline)))
headline[:5]

노이즈 제거 전: 1324
노이즈 제거 후: 1214


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

다음으로 학습의 효율을 위해 구두점을 제거하고 모든 문자를 소문자로 변경해주자

In [17]:
def repreprocessing(s):
    s = s.encode("utf8").decode("ascii", 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

## 단어 집합 만들기

In [18]:
t_lstm = Tokenizer()
t_lstm.fit_on_texts(text)
vocab_size_lstm = len(t_lstm.word_index)+1
print('LSTM 학습용 단어 집합의 크기: {}'.format(vocab_size_lstm))

LSTM 학습용 단어 집합의 크기: 3494


In [19]:
sequences = []
for line in text:
    encoded = t_lstm.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print(sequences[:11])

[[99, 269], [99, 269, 371], [99, 269, 371, 1115], [99, 269, 371, 1115, 582], [99, 269, 371, 1115, 582, 52], [99, 269, 371, 1115, 582, 52, 7], [99, 269, 371, 1115, 582, 52, 7, 2], [99, 269, 371, 1115, 582, 52, 7, 2, 372], [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10], [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116], [100, 3]]


가독성을 위해 단어-정수 변환 사전을 만들어놓자

In [20]:
index_to_word = {}
for key, value in t_lstm.word_index.items():
    index_to_word[value] = key
print('No.3: {}'.format(index_to_word[3]))
print('No.5: {}'.format(index_to_word[5]))

No.3: to
No.5: in


이제 전체 샘플 길이를 동일하게 만들고 y를 분리해내자

In [24]:
# 최대 길이 측정
max_len_lstm = max(len(l) for l in sequences)
print('샘플의 최대 길이: {}'.format(max_len_lstm))

# 패딩
sequences = pad_sequences(sequences, maxlen=max_len_lstm, padding='pre')
print('sequences 예시')
print(sequences[:3])

# y 분리
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size_lstm)
print('X, y 예시')
print(X[:3])
print(y[:3])

샘플의 최대 길이: 24
sequences 예시
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]
X, y 예시
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 모델 설계하기

각 단어의 임베딩 벡터는 10차원으로, 은닉 상태는 128개로 하는 LSTM 모델을 생성한다.

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [25]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size_lstm, 10, input_length=max_len_lstm-1))
lstm_model.add(LSTM(128))
lstm_model.add(Dense(vocab_size_lstm, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
 - 15s - loss: 7.6518 - acc: 0.0261
Epoch 2/100
 - 17s - loss: 7.1480 - acc: 0.0282
Epoch 3/100
 - 14s - loss: 7.0591 - acc: 0.0311
Epoch 4/100
 - 17s - loss: 6.9901 - acc: 0.0373
Epoch 5/100
 - 18s - loss: 6.8921 - acc: 0.0433
Epoch 6/100
 - 16s - loss: 6.7461 - acc: 0.0470
Epoch 7/100
 - 17s - loss: 6.5661 - acc: 0.0499
Epoch 8/100
 - 17s - loss: 6.3809 - acc: 0.0519
Epoch 9/100
 - 18s - loss: 6.1941 - acc: 0.0557
Epoch 10/100
 - 16s - loss: 6.0120 - acc: 0.0598
Epoch 11/100
 - 16s - loss: 5.8407 - acc: 0.0647
Epoch 12/100
 - 15s - loss: 5.6763 - acc: 0.0709
Epoch 13/100
 - 12s - loss: 5.5202 - acc: 0.0746
Epoch 14/100
 - 11s - loss: 5.3713 - acc: 0.0784
Epoch 15/100
 - 13s - loss: 5.2268 - acc: 0.0784
Epoch 16/100
 - 11s - loss: 5.0830 - acc: 0.0883
Epoch 17/100
 - 11s - loss: 4.9420 - acc: 0.0943
Epoch 18/100
 - 12s - loss: 4.8038 - acc: 0.1080
Epoch 19/100
 - 11s - loss: 4.6638 - acc: 0.1188
Epoch 20/100
 - 11s - loss: 4.5205 - acc: 0.1382
Epoch 21/100
 - 12s - loss: 4

문장을 생성해보고 학습이 잘 되었는지 확인해보자.

In [29]:
print(sentence_generation(lstm_model, t_lstm, 'i', 10, max_len_lstm))
print(sentence_generation(lstm_model, t_lstm, 'how', 10, max_len_lstm))

i want to be rich and im not sorry it best
how to make facebook more accountable gluten and gop at say
